In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot
from wordcloud import WordCloud, STOPWORDS 
from warnings import filterwarnings
filterwarnings('ignore')
import missingno as msno
import glob

In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    district_id = filename.split("/")[4].split(".")[0]
    df["district_id"] = district_id
    li.append(df)
    
engagement_df = pd.concat(li)
engagement_df = engagement_df.reset_index(drop=True)
engagement_df.head()

In [ ]:
districts_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
districts_df.head()

In [ ]:
products_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
products_df.head()

In [ ]:
products_df.info()

In [ ]:
products_df.isnull().sum()

In [ ]:
products_df.columns

In [ ]:
products_df.shape

In [ ]:
msno.bar(products_df, figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
msno.bar(engagement_df, figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
msno.bar(districts_df, figsize=(10,5), fontsize=12)
plt.show()

In [ ]:
districts_df.isnull().sum()

In [ ]:
engagement_df.isnull().sum()

In [ ]:
districts_df.duplicated().sum()

In [ ]:
engagement_df.duplicated().sum()

In [ ]:
products_df.duplicated().sum()

In [ ]:
locations=pd.DataFrame({"Name":districts_df['state'].unique()})

In [ ]:
geolocator=Nominatim(user_agent="app")
lat=[]
lon=[]
for location in locations['Name']:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

In [ ]:
locations['lat']=lat
locations['lon']=lon

In [ ]:
locations.head()

In [ ]:
Rest_locations=pd.DataFrame(districts_df['state'].value_counts().reset_index())

In [ ]:
Rest_locations.columns=['Name','count']
final_loc=Rest_locations.merge(locations,on='Name',how="left").dropna()
final_loc.head()

In [ ]:
def generateBaseMap(default_location=[37.0902, -95.7129], default_zoom_start=4):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

In [ ]:
basemap=generateBaseMap()

In [ ]:
HeatMap(final_loc[['lat','lon','count']],zoom=20,radius=20).add_to(basemap)

In [ ]:
basemap

In [ ]:
FastMarkerCluster(data=final_loc[['lat','lon','count']].values.tolist()).add_to(basemap)

basemap

In [ ]:
labels = list(districts_df.locale.value_counts().index)
values = districts_df['state'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="Locale Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()

In [ ]:
labels = list(districts_df.state.value_counts().index)
values = districts_df['state'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="State Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()

In [ ]:
labels = list(products_df['Sector(s)'].value_counts().index)
values = products_df['Sector(s)'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="Sector Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()